In [83]:
import numpy as np 
import pandas as pd
#import calendar
import math

# plots
import matplotlib.pyplot as plt
#import seaborn as sns
%matplotlib inline

In [84]:
filenames = ['properties/properati-AR-2016-03-01-properties-sell-six_months.csv',
            'properties/properati-AR-2016-04-01-properties-sell.csv',
            'properties/properati-AR-2016-05-01-properties-sell.csv',
            'properties/properati-AR-2016-06-01-properties-sell.csv',
            'properties/properati-AR-2016-07-01-properties-sell.csv',
            'properties/properati-AR-2016-08-01-properties-sell.csv',
            'properties/properati-AR-2016-09-01-properties-sell.csv',
            'properties/properati-AR-2016-10-01-properties-sell.csv',
            'properties/properati-AR-2016-11-01-properties-sell.csv',
            'properties/properati-AR-2016-12-01-properties-sell.csv', 
            'properties/properati-AR-2017-01-01-properties-sell.csv',
            'properties/properati-AR-2017-02-01-properties-sell.csv',
            'properties/properati-AR-2017-08-01-properties-sell-six_months.csv']

dataframes = []
for filename in filenames:
    dataframes.append(pd.read_csv(filename))
    
#Uno los dataFrame
metadata = pd.DataFrame()
for dataframe in dataframes:
    metadata = pd.concat([metadata, dataframe]).drop_duplicates().reset_index(drop=True)

In [85]:
data = metadata.loc[(metadata.state_name.str.contains('Buenos Aires')) | (metadata.state_name.str.contains\
                ('Capital Federal'))  | (metadata.state_name.str.contains\
                ('Bs.As') ),:]
data.shape
# 1.283.741 entries antes, ahora 242290

(242290, 27)

In [86]:
test = pd.read_csv('properties/properati_dataset_testing_noprice.csv')
test_visualiz = test.loc[:,['state_name', 'id']].groupby('state_name').agg(np.size)
test_visualiz.head(10)

,id
state_name,
Bs.As. G.B.A. Zona Norte,2
Bs.As. G.B.A. Zona Oeste,3101
Bs.As. G.B.A. Zona Sur,2215
Buenos Aires Interior,1
Capital Federal,8847


In [87]:
data['state_name'].tail()

296472                 Capital Federal
296473        Bs.As. G.B.A. Zona Norte
296474                 Capital Federal
296475    Buenos Aires Costa Atlántica
296476                 Capital Federal
Name: state_name, dtype: object

In [88]:
# Depuración del set de test
#Completo los valores NaN con promedios para realizar la predicción
prom_surface_total_in_m2 = test['surface_total_in_m2'].mean()
prom_surface_covered_in_m2 = test['surface_covered_in_m2'].mean()
prom_rooms = int(test['rooms'].mean())
prom_lat = test['lat'].mean()
prom_lon = test['lon'].mean()

test_float = test
test_float['surface_total_in_m2'].fillna(prom_surface_total_in_m2, inplace=True)
test_float['surface_covered_in_m2'].fillna(prom_surface_covered_in_m2, inplace=True)
test_float['rooms'].fillna(prom_rooms, inplace=True)
test_float['lat'].fillna(prom_lat, inplace=True)
test_float['lon'].fillna(prom_lon, inplace=True)


test_float.head(100)

,id,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description
0,3632,2017-08-24,departamento,venta,Puerto Madero,|Argentina|Capital Federal|Puerto Madero|,Argentina,Capital Federal,"-34.6109877599,-58.3634635778",-34.610988,-58.363464,0.000000,359.471588,NaN,2.0,NaN,Edificio BA Houses situado frente al Dique 3 d...
1,3633,2017-08-25,departamento,venta,Buenos Aires Interior,|Argentina|Buenos Aires Interior|,Argentina,Buenos Aires Interior,NaN,-34.629923,-58.465820,0.000000,359.471588,NaN,2.0,NaN,El departamento cuenta con un living-comedor a...
2,2263404,2017-08-01,departamento,venta,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,"-34.5893633232,-58.4128798588",-34.589363,-58.412880,53.000000,48.000000,NaN,2.0,1500,IMPECABLE TORRE COY III – DEPA...
3,2263405,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,39.000000,39.000000,NaN,2.0,NaN,AMBIENTE DIVISIBLE CON PISOS D...
4,2263406,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,51.000000,51.000000,NaN,2.0,NaN,LIVING COMEDOR CON PISOS DE PO...
5,2263407,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,39.000000,39.000000,NaN,2.0,NaN,AMBIENTE DIVISIBLE CON PISOS D...
6,2263408,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,39.000000,39.000000,NaN,2.0,NaN,AMBIENTE DIVISIBLE CON PISOS D...
7,2263409,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,39.000000,39.000000,NaN,2.0,NaN,AMBIENTE DIVISIBLE CON PISOS D...
8,2263410,2017-08-01,departamento,venta,Colegiales,|Argentina|Capital Federal|Colegiales|,Argentina,Capital Federal,"-34.5787213694,-58.4570952614",-34.578721,-58.457095,67.000000,67.000000,NaN,2.0,2600,EXCELENTE 2 AMB- APTO PROFESIO...
9,2263411,2017-08-01,departamento,venta,Almagro,|Argentina|Capital Federal|Almagro|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,47.000000,47.000000,NaN,2.0,NaN,IMPECABLE MONOAMBIENTE- DIVISI...


In [89]:
test_float.count()

id                         14166
created_on                 14166
property_type              14166
operation                  14166
place_name                 14166
place_with_parent_names    14166
country_name               14166
state_name                 14166
lat-lon                    10487
lat                        14166
lon                        14166
surface_total_in_m2        14166
surface_covered_in_m2      14166
floor                       1368
rooms                      14166
expenses                    2543
description                14166
dtype: int64

<br><B><FONT FACE="Arial" SIZE="5">Prediccion usando XGBoosting'</FONT></B><br/>

In [90]:
#FILTRO LOS DATOS ANOMALOS

data_float = data.loc[:, ['price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',\
                          'rooms', 'lat', 'lon']].dropna()


In [91]:

#filtro
data_float = \
data_float.loc[((data_float.surface_total_in_m2 >= 0.0) & (data_float.surface_total_in_m2 <= 1000000.0)) & \
               ((data_float.surface_covered_in_m2 >= 0.0) & (data_float.surface_covered_in_m2 <= 50000000.0)) & \
               ((data_float.rooms >= 0.0) & (data_float.rooms <= 200.0)), :]


data_float.shape
#(68586, 7)

(68586, 6)

In [92]:
#Creo X e Y
feature_cols = ['surface_total_in_m2', 'surface_covered_in_m2', 'rooms']
TRAIN = data_float[:2600]
TEST = data_float[2600:]

#Separo en set de entrenamiento y set de Test
X_train = TRAIN[feature_cols]
X_test = TEST[feature_cols]

y_train = TRAIN.price_aprox_usd
y_test = TEST.price_aprox_usd

In [93]:
#Entreno el modelo
from sklearn import ensemble
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls', 'criterion': 'mse'}

params2 = {'random_state': 0, 'n_estimators': 500,
                                  'max_depth': 1, 'learning_rate': 0.01}
clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=4, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=500,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [94]:
#Me fijo la precision
#random_state=0, n_estimators=500, max_depth=1, learning_rate=0.01
print('Precisión Gradient Boosting train/test  {0:.3f}/{1:.3f}'
      .format(clf.score(X_train, y_train), clf.score(X_test, y_test)))

#Precisión Gradient Boosting train/test  0.838/0.717
#Precisión Gradient Boosting train/test  0.686/0.325 cuando la precisión era mejor, nuestras predicciones eran 
#peores. Habría que ver cómo mide la precisión ésta cosa

Precisión Gradient Boosting train/test  0.686/0.325


In [95]:
#Hago la prediccion para Kaggle
test['price_aprox_usd_XGB'] = clf.predict(test.loc[:, ['surface_total_in_m2', 'surface_covered_in_m2',\
                                        'rooms']])
test['price_aprox_usd_XGB'].head(100)

0     515041.601125
1     515041.601125
2      90679.766538
3      81078.047436
4     116454.299722
5      81078.047436
6      81078.047436
7      81078.047436
8     116516.010293
9      90679.766538
10     90679.766538
11     90679.766538
12    116910.977904
13    228130.918058
14     81078.047436
15     81078.047436
16     81078.047436
17     81078.047436
18     81078.047436
19     81078.047436
20     81078.047436
21     81078.047436
22    106525.807081
23    116454.299722
24    116454.299722
25    116454.299722
26    116454.299722
27     81078.047436
28     81078.047436
29     81078.047436
          ...      
70    116910.977904
71    102337.898510
72    180907.533401
73    180907.533401
74    116910.977904
75    116910.977904
76    116910.977904
77    116910.977904
78    190314.922531
79    116910.977904
80    161983.035448
81    190314.922531
82    209874.787030
83    168115.059952
84    167936.601949
85     90679.766538
86    192776.517821
87     81246.952735
88    155665.732310


In [96]:
#Genero el csv con los nombres de columnas requeridos por Kaggle
#test_float_reduced = test_float.loc[:,['id', 'price_aprox_usd_XGB']]
#test_float_reduced.columns=['id', 'price_usd']
#test_float_reduced.to_csv('SubmitXGBoostingBuenosAires.csv', index=False)

<br><B><FONT FACE="Arial" SIZE="5">XGBoosting con lat y lon</FONT></B><br/>

In [97]:
#Creo X e Y
feature_cols = ['surface_total_in_m2', 'surface_covered_in_m2', 'rooms', 'lat', 'lon']
TRAIN = data_float[:2600]
TEST = data_float[2600:]

#Separo en set de entrenamiento y set de Test
X_train = TRAIN[feature_cols]
X_test = TEST[feature_cols]

y_train = TRAIN.price_aprox_usd
y_test = TEST.price_aprox_usd

In [98]:
#Entreno el modelo
from sklearn import ensemble
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}

params2 = {'random_state': 0, 'n_estimators': 500,
                                  'max_depth': 1, 'learning_rate': 0.01}
clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=4, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=500,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [99]:
#Me fijo la precision
#random_state=0, n_estimators=500, max_depth=1, learning_rate=0.01
print('Precisión Gradient Boosting train/test  {0:.3f}/{1:.3f}'
      .format(clf.score(X_train, y_train), clf.score(X_test, y_test)))

#Precisión Gradient Boosting train/test  0.905/0.736
#Precisión Gradient Boosting train/test  0.864/0.549

Precisión Gradient Boosting train/test  0.864/0.549


In [100]:
#Hago la prediccion para Kaggle
test['price_aprox_usd_XGB'] = clf.predict(test.loc[:, ['surface_total_in_m2', 'surface_covered_in_m2',\
                                        'rooms',  'lat', 'lon']])

#Genero el csv con los nombres de columnas requeridos por Kaggle
test_float_reduced = test_float.loc[:,['id', 'price_aprox_usd_XGB']]
test_float_reduced.columns=['id', 'price_usd']
#test_float_reduced.to_csv('SubmitXGBoostingLatLonGenerico.csv', index=False)
test_float_reduced['price_usd'].head(100)

0     958515.672248
1     481911.824986
2     105699.924684
3      70874.893620
4      97398.849546
5      70874.893620
6      70874.893620
7      70874.893620
8     140056.138198
9      81255.889411
10     81255.889411
11     81255.889411
12    106648.755998
13    437813.718234
14     70874.893620
15     70874.893620
16     70874.893620
17     70874.893620
18     70874.893620
19     70874.893620
20     70874.893620
21     70874.893620
22     91873.829572
23     97398.849546
24     97398.849546
25     97398.849546
26     97398.849546
27     70874.893620
28     70874.893620
29     91362.177433
          ...      
70    106648.755998
71     72310.873565
72    130579.360960
73    130579.360960
74    106648.755998
75    106648.755998
76    106648.755998
77    106648.755998
78    314184.094027
79    106648.755998
80    125091.578876
81    314184.094027
82    129673.107564
83    122020.475251
84    209483.970977
85     81255.889411
86    285095.558551
87     96988.954038
88    125091.578876


<br><B><FONT FACE="Arial" SIZE="5">Uso grid-search para estimar los parametros</FONT></B><br/>

In [101]:
#Creo X e Y
feature_cols = ['surface_total_in_m2', 'surface_covered_in_m2', 'rooms',  'lat', 'lon']
TRAIN = data_float[:2600]
TEST = data_float[2600:]

#Separo en set de entrenamiento y set de Test
X_train = TRAIN[feature_cols]
X_test = TEST[feature_cols]

y_train = TRAIN.price_aprox_usd
y_test = TEST.price_aprox_usd

In [102]:
#Grid-search para encontrar el parametro n_estimators
#Arranco con parametros normales al azar.
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

param_test1 = {'n_estimators':list(range(1,600,10))}

gsearch1 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, min_samples_split=500,\
min_samples_leaf=50, max_depth=8, max_features='sqrt', subsample=0.8, random_state=10),\
param_grid = param_test1, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)

#gsearch1 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1 ,max_features='sqrt',\
#subsample=0.8, random_state=10), param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
                        
gsearch1.fit(X_train, y_train)
#Me fijo el resultado del parametro

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=8,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=50,
             min_samples_split=500, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=10,
             subsample=0.8, verbose=0, warm_start=False),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'n_estimators': [1, 11, 21, 31, 41, 51, 61, 71, 81, 91, 101, 111, 121, 131, 141, 151, 161, 171, 181, 191, 201, 211, 221, 231, 241, 251, 261, 271, 281, 291, 301, 311, 321, 331, 341, 351, 361, 371, 381, 391, 401, 411, 421, 431, 441, 451, 461, 471, 481, 491, 501, 511, 521, 531, 541, 551, 561, 571, 581, 591]},
       pre_dispatch='2*n_jobs', refit=True,
       scoring='neg_mean_squared_error', verbose=0)

In [103]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_


([mean: -69234945870.96466, std: 40984584818.28320, params: {'n_estimators': 1},
  mean: -53081446078.64149, std: 37321494542.32595, params: {'n_estimators': 11},
  mean: -48326934790.11139, std: 35805634722.60236, params: {'n_estimators': 21},
  mean: -45795188936.93011, std: 34061743761.00172, params: {'n_estimators': 31},
  mean: -45250608488.26923, std: 33276120017.21741, params: {'n_estimators': 41},
  mean: -44271960339.86526, std: 32695131872.42344, params: {'n_estimators': 51},
  mean: -43809599364.82716, std: 31664582621.68012, params: {'n_estimators': 61},
  mean: -43221131888.34174, std: 30674313618.28349, params: {'n_estimators': 71},
  mean: -42704764854.36606, std: 30140851082.67236, params: {'n_estimators': 81},
  mean: -42062400397.78432, std: 29626962904.08623, params: {'n_estimators': 91},
  mean: -41701803383.33879, std: 29405085344.51485, params: {'n_estimators': 101},
  mean: -41410201347.08528, std: 28948192387.41150, params: {'n_estimators': 111},
  mean: -415428

In [104]:
#Encontrar los parametros max_depth y min_samples_split
#Aca ya uso n_estimators=491
param_test2 = {'max_depth':list(range(2,16,2)), 'min_samples_split':list(range(200,1001,10))}
gsearch2 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, n_estimators=491, max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_test2, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)
gsearch2.fit(X_train,y_train)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

([mean: -48519329606.29023, std: 26850424857.88991, params: {'min_samples_split': 200, 'max_depth': 2},
  mean: -48357439308.00492, std: 26809255555.66475, params: {'min_samples_split': 210, 'max_depth': 2},
  mean: -48242707678.03746, std: 28466495446.28462, params: {'min_samples_split': 220, 'max_depth': 2},
  mean: -48214987083.18517, std: 28953762015.33075, params: {'min_samples_split': 230, 'max_depth': 2},
  mean: -47152991219.15722, std: 28204668545.77505, params: {'min_samples_split': 240, 'max_depth': 2},
  mean: -48411326059.42489, std: 27954342190.83569, params: {'min_samples_split': 250, 'max_depth': 2},
  mean: -48380514718.31116, std: 28327508509.66961, params: {'min_samples_split': 260, 'max_depth': 2},
  mean: -48292071069.59199, std: 28998751162.71653, params: {'min_samples_split': 270, 'max_depth': 2},
  mean: -47768386253.05447, std: 27775815170.60978, params: {'min_samples_split': 280, 'max_depth': 2},
  mean: -47724210851.91771, std: 28213934540.38611, params: {'mi

In [105]:
#Encontrar el parametro min_samples_leaf
# 'n_estimators': 491, 'max_depth': 12, 'min_samples_split': 210

param_test3 = {'min_samples_leaf':list(range(10,100,1))}
gsearch3 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, n_estimators=491, min_samples_split=210, max_depth=12,max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_test3, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)
gsearch3.fit(X_train, y_train)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

([mean: -40588288759.21691, std: 20229215058.92402, params: {'min_samples_leaf': 10},
  mean: -40968288067.12892, std: 20192862042.07670, params: {'min_samples_leaf': 11},
  mean: -41660975542.53265, std: 20335674865.45102, params: {'min_samples_leaf': 12},
  mean: -41091495667.23310, std: 20437942493.84439, params: {'min_samples_leaf': 13},
  mean: -39725368942.67793, std: 20986501160.70427, params: {'min_samples_leaf': 14},
  mean: -40640572424.11992, std: 22039352672.88519, params: {'min_samples_leaf': 15},
  mean: -40315702967.15980, std: 20888299830.18079, params: {'min_samples_leaf': 16},
  mean: -40273756118.16009, std: 21085210322.99110, params: {'min_samples_leaf': 17},
  mean: -40787215360.45503, std: 20665658455.94897, params: {'min_samples_leaf': 18},
  mean: -39274449959.37663, std: 20938272061.03363, params: {'min_samples_leaf': 19},
  mean: -39776948805.70848, std: 20436806832.73470, params: {'min_samples_leaf': 20},
  mean: -39271340207.26553, std: 21244067511.67280, pa

In [106]:
#Encontrar el parametro max_features
# 'n_estimators': 491, 'max_depth': 12, 'min_samples_split': 210, 'min_samples_leaf': 65

param_test4 = {'max_features':list(range(1,6,1))}
gsearch4 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, n_estimators=491, max_depth=12, min_samples_split=210, min_samples_leaf=65, subsample=0.8, random_state=10),
param_grid = param_test4, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)
gsearch4.fit(X_train, y_train)
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

([mean: -37384880074.18290, std: 21926069674.30249, params: {'max_features': 1},
  mean: -37113144650.02412, std: 20079866790.82848, params: {'max_features': 2},
  mean: -38097886060.76998, std: 20609056942.60023, params: {'max_features': 3},
  mean: -37043814926.80031, std: 20540278196.32262, params: {'max_features': 4},
  mean: -38306897957.03674, std: 20547598898.55463, params: {'max_features': 5}],
 {'max_features': 4},
 -37043814926.80031)

In [107]:
# 'n_estimators': 491, 'max_depth': 12, 'min_samples_split': 210, 'min_samples_leaf': 65, 'max_features': 4
#Voy a realizar una prueba con estos parametros
#Entreno el modelo
from sklearn import ensemble
params = {'n_estimators': 491, 'max_depth': 12, 'min_samples_split': 210, 'min_samples_leaf':65 , 'max_features': 4,
          'learning_rate': 0.1}


clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(X_train, y_train)

#Me fijo la precision
print('Precisión Gradient Boosting train/test  {0:.3f}/{1:.3f}'
      .format(clf.score(X_train, y_train), clf.score(X_test, y_test)))

#Precisión Gradient Boosting train/test  0.823/0.767
#Precisión Gradient Boosting train/test  0.875/0.526

Precisión Gradient Boosting train/test  0.875/0.526


In [108]:
#Hago la prediccion para Kaggle
test['price_aprox_usd_XGB'] = clf.predict(test.loc[:, ['surface_total_in_m2', 'surface_covered_in_m2',\
                                        'rooms',  'lat', 'lon']])

#Genero el csv con los nombres de columnas requeridos por Kaggle
test_float_reduced = test_float.loc[:,['id', 'price_aprox_usd_XGB']]
test_float_reduced.columns=['id', 'price_usd']
test_float_reduced.to_csv('SubmitXGBoostingGS.csv', index=False)

## Uso grid-search para estimar los parametros, ésta vez todos juntos


In [ ]:
#Creo X e Y
#feature_cols = ['surface_total_in_m2', 'surface_covered_in_m2', 'floor', 'rooms',  'lat', 'lon']
#TRAIN = data_float[:2600]
#TEST = data_float[2600:]

#Separo en set de entrenamiento y set de Test
#X_train = TRAIN[feature_cols]
#X_test = TEST[feature_cols]

#y_train = TRAIN.price_aprox_usd
#y_test = TEST.price_aprox_usd

In [ ]:
#Grid-search para encontrar el parametro n_estimators
#Arranco con parametros normales al azar.
#from sklearn.grid_search import GridSearchCV
#from sklearn.ensemble import GradientBoostingRegressor

#param_test1 = {'n_estimators':list(range(1,500,5)), 'max_depth':list(range(2,16,2)),\
#               'min_samples_split':list(range(200,1000,50)), 'min_samples_leaf':list(range(10,100,5)),\
#                'max_features':list(range(1,4,1))}

#gsearch1 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, subsample=0.8, random_state=10),\
                param_grid = param_test1, scoring='neg_mean_squared_error', n_jobs=4,iid=False, cv=5)
                        
#gsearch1.fit(X_train, y_train)

In [ ]:
# {'max_depth': 10,
#  'max_features': 3,
#  'min_samples_leaf': 40,
#  'min_samples_split': 200,
#  'n_estimators': 261},

#gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

### Con estos hiper-parametros estimados, hago una prediccion

In [ ]:
# 'n_estimators':261, 'max_depth': 10, 'min_samples_split': 200, 'min_samples_leaf': 40, 'max_features': 3
#Voy a realizar una prueba con estos parametros
#Entreno el modelo
#from sklearn import ensemble
#params = {'n_estimators': 261, 'max_depth': 10, 'min_samples_split': 200, 'min_samples_leaf': 40, 'max_features': 3,
#          'learning_rate': 0.1}


#clf = ensemble.GradientBoostingRegressor(**params)

#clf.fit(X_train, y_train)

#Me fijo la precision
#print('Precisión Gradient Boosting train/test  {0:.3f}/{1:.3f}'
#      .format(clf.score(X_train, y_train), clf.score(X_test, y_test)))

#Precisión Gradient Boosting train/test  0.857/0.837

In [ ]:
#Hago la prediccion para Kaggle
test['price_aprox_usd_XGB'] = clf.predict(test.loc[:, ['surface_total_in_m2', 'surface_covered_in_m2',\
                                        'floor', 'rooms',  'lat', 'lon']])

#Genero el csv con los nombres de columnas requeridos por Kaggle
test_float_reduced = test_float.loc[:,['id', 'price_aprox_usd_XGB']]
test_float_reduced.columns=['id', 'price_usd']
test_float_reduced.to_csv('SubmitXGBoostingGS2.csv', index=False)